In [ ]:
import pygsheets
import pandas as pd
import numpy as np
import requests
import io
import csv
from google.oauth2 import service_account
import pandas_gbq
import json 
import xmltodict
import os

In [ ]:
START_DATE = (pd.to_datetime('today') - pd.Timedelta(days=3)).strftime('%Y-%m-%d')
END_DATE = pd.to_datetime('today').strftime('%Y-%m-%d')

In [ ]:
print("date_from: ", START_DATE)
print("date_to: ", END_DATE)

In [ ]:
CURRENCIES = 'EUR,AED,CNY'

CURRENCIES = CURRENCIES.replace(' ', '').split(',')
# всегда добавляем доллар, тк по нему проверяем работала ли биржа
if 'USD' not in CURRENCIES:
    CURRENCIES.append('USD')
CURRENCIES

In [ ]:
def get_currency_codes(currency:list) -> dict:
    """
    This function retrieves currency codes given as a list of currency symbols.

    Parameters:
    currency (list): a list of currency symbols (strings)

    Returns:
    dictionary: a mapping of currency symbols to codes
    """
    
    url = "https://www.cbr-xml-daily.ru/daily.xml"
    responce = requests.get("https://www.cbr-xml-daily.ru/daily.xml")
    data_dict = xmltodict.parse(responce.content)
    data_dict = data_dict["ValCurs"]["Valute"]

    df_data = pd.DataFrame()
    for item in data_dict:
        df_tmp = pd.DataFrame([item])
        df_data = pd.concat([df_data, df_tmp])
        
    df_data = df_data[['@ID', 'CharCode']]
    df_data.columns = ['Code', 'Currency']
    df_data = df_data[df_data['Currency'].isin(currency)]
    codes = df_data.set_index('Currency')['Code'].to_dict()

    return codes

get_currency_codes(CURRENCIES)

In [ ]:
def get_currency_rate(
    start_date: str, end_date: str, currency_codes: dict
) -> pd.DataFrame:

    # format dates for api
    start_date = pd.to_datetime(start_date).strftime("%d/%m/%Y")
    end_date = pd.to_datetime(end_date).strftime("%d/%m/%Y")

    params = {
            "date_req1": start_date,
            "date_req2": end_date,
        }

    df_rates = pd.DataFrame()
    for currency_name, currency_code in currency_codes.items():
        print(f"currency: {currency_name}, code: {currency_code}, start: {start_date}, end: {end_date}")
        
        params["VAL_NM_RQ"] = currency_code
        req = response = requests.get(
            f"http://www.cbr.ru/scripts/XML_dynamic.asp", params=params
        )

        if req.status_code == 200:
            df = pd.read_xml(io.StringIO(req.text))
            if df.empty:
                print('receiced no data - create empty df')
                df_tmp = pd.DataFrame(columns=['business_dt', 'currency', 'rate', 'rate_changed'])
            else:
                print('receiced data')
                df["currency"] = currency_name.lower()
                df['rate_changed'] = True
                df = df[["Date", "currency", "Value", 'rate_changed']]
                df.columns = ["business_dt", "currency",  "rate", 'rate_changed']
            
            df_rates = pd.concat([df_rates, df], ignore_index=True)
            print('success', end='\n\n')
        
        else: 
            print(req.status_code, 'error', sep=' - ')
            print(req.text)
            break

    # add rub
    df_rub = pd.DataFrame(columns=['business_dt', 'currency', 'rate', 'rate_changed'])
    df_rub['business_dt'] = df_rates['business_dt']
    df_rub['currency'] = 'rub'
    df_rub['rate'] = '1,0'
    df_rub['rate_changed'] = True
    df_rates = pd.concat([df_rates, df_rub], ignore_index=True)

    df_rates['business_dt'] = pd.to_datetime(df_rates['business_dt'], format='%d.%m.%Y')

    df_rates.sort_values(by=['business_dt', 'currency'], inplace=True)
    df_rates.reset_index(drop=True, inplace=True)

    if pd.to_datetime(start_date, format='%d/%m/%Y') not in df_rates['business_dt'].values:
        start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
        start_date = start_date - pd.Timedelta(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
        end_date = pd.to_datetime(end_date, format='%d/%m/%Y').strftime('%Y-%m-%d')
        
        df_rates = get_currency_rate(start_date, end_date, currency_codes)

    return df_rates

In [ ]:
def get_all_dates(df: pd.DataFrame, end_date: str) -> pd.DataFrame:
    min_date = df['business_dt'].min()
    # max_date = df['business_dt'].max()
    max_date = pd.to_datetime(end_date)
    currencies = df['currency'].unique()

    df_dates = pd.DataFrame()
    for currency in currencies:
        df_tmp = pd.DataFrame({'business_dt': pd.date_range(min_date, max_date)})
        df_tmp['currency'] = currency
        df_dates = pd.concat([df_dates, df_tmp], ignore_index=True)

    df_rates = pd.merge(df_dates, df, on=['business_dt', 'currency'], how='left')
    df_rates = df_rates.sort_values(by=['currency', 'business_dt']).reset_index(drop=True)
    df_rates['rate_changed'] = df_rates['rate_changed'].fillna(False)
    df_rates['rate'] = df_rates['rate'].ffill()

    return df_rates

In [ ]:
def correct_formats(df: pd.DataFrame) -> pd.DataFrame:
    df['business_dt'] = pd.to_datetime(df['business_dt'], format='%Y-%m-%d')
    df['currency'] = df['currency'].astype(str)
    df['rate'] = df['rate'].str.replace(',', '.').astype(float)
    df['rate_changed'] = df['rate_changed'].astype(bool)
    return df

In [ ]:
rates_from_api = get_currency_rate(START_DATE, END_DATE, get_currency_codes(CURRENCIES))
rates_dates_filled = get_all_dates(rates_from_api, pd.to_datetime(END_DATE))
rates = correct_formats(rates_dates_filled)

In [ ]:
start_clear = rates['business_dt'].min().strftime('%Y-%m-%d')
end_clear = rates['business_dt'].max().strftime('%Y-%m-%d')

print('clear from: ', start_clear, sep='\t')
print('clear to: ', end_clear, sep='\t')

In [ ]:
info = json.loads(os.environ["BQ"])
creds = service_account.Credentials.from_service_account_info(info)

delete_query = f"""
DELETE FROM `schema`.`currency_daily`
WHERE business_dt BETWEEN ({start_clear}) AND ({end_clear});
"""

# Execute the delete query using pandas-gbq
pd.io.gbq.read_gbq(delete_query, project_id='project_id', credentials=creds)

In [ ]:
info = json.loads(os.environ["BQ"])
creds = service_account.Credentials.from_service_account_info(info)

# df_fraud = df_fraud.astype(str)
pandas_gbq.to_gbq(
    rates,
    "schema.currency_daily",
    project_id="dwh-r10",
    if_exists="append",
    credentials=creds,
    table_schema=[{"name": "business_dt", "type": "DATE"}],
)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a143e142-8b95-448d-a997-d9a2c97e85fd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>